# Wordle?

> Any CS student should be able to create a wordle solver.

I've lost track of how many said this on Twitter and I completely agree. But as [1blue3brown](https://youtu.be/v68zYyaEmEA) showed in a recent video, there is a information-theoretic upper limit to what the best wordle solver can score, and it generally boils down to the probabilities between the frequency of the particular word appearing in the language and the wordle games selection of words.

But what if all probabilities are equal?

Let's ask the computer to answer this question...

In [1]:
import pathlib
import random
words = pathlib.Path('wordle_words.txt').read_text('utf-8').split('\n')

def play():
    print("(q) to quit, (r) for random word")
    ans = None
    while ans != 'q':
        ans = input("Enter the secret word >>>")
        if ans == 'r':
            ans = random.choice(words)
        elif ans == 'q':
            break
        
        if ans not in words:
            print(f"{ans} is not an english 5 letter word")
        else:
            autowordle(secret_word=ans)

In [2]:
def compare(guess, secret):
    L = list(guess.lower())
    # compare:
    a,b = {},{}
    for ix,(A,B) in enumerate(zip(guess, secret)):
        if A == B:
            L[ix] = A
        elif A not in secret:
            L[ix] = '-'
        else:
            pass
        a[A] = 1 if A not in a else a[A]+1  # momma --> {'m': 3, ...}
        b[B] = 1 if B not in b else b[B]+1  # forma --> {'m':1, ...}

    for ix, A in enumerate(L):
        if A == '-': continue
        if A == secret[ix]: continue

        if a[A.upper()] > b.get(A.upper(),0):
            a[A.upper()] -= 1
            L[ix] = '-'
    return "".join(L)

In [3]:
compare('MOMMA', 'FORMA')

'-O-MA'

In [4]:
def could_match(guess, word):
    # hard match
    for ix,c in enumerate(guess):
        if c == '-':
            continue
        if c.isupper():
            if word[ix] != c:  # the letter is upper and NOT in right position.
                return False
            else:  # if c.isupper() and word[ix] == c: 
                continue
        if c.islower():  # lowercase letter in the exact position IS a match
            if word[ix] == c.upper():
                continue
            if guess.count(c.upper()) > word.count(c.upper()):
                return False
    return True

In [5]:
assert could_match('-O-MA', "FORMA")

In [6]:
def autowordle(secret_word):
    # computer tries to guess.
    options = words[:]
    eliminate, keep = set(), set()
    guesses = 0
    while 1:
        guess = random.choice(options)
        guesses += 1
        if guess == secret_word:
            print(guesses, ":", f"{guess} is correct!")
            break
        options.remove(guess)

        word = compare(guess, secret_word)
        
        keep.update({c.upper() for c in word if c != '-'})
        eliminate.update({c for c in guess if c not in keep})

        options = [w for w in options if not any(c in w for c in eliminate)]
        if keep:
            options = [w for w in options if keep.issubset(set(w))]
        options = [w for w in options if could_match(word,w)]
                
        print(guesses,":", guess, "-->", word)
    return guesses

In [7]:
autowordle('SWIFT')

1 : NODUS --> ----s
2 : CHAMS --> ----s
3 : VIBES --> -i--s
4 : TIPIS --> t--is
5 : FISTS --> fi-ts
6 : LIFTS --> -ifts
7 : FRITS --> f-Its
8 : SWIFT is correct!


8

In [8]:
play()

(q) to quit, (r) for random word
1 : DOYST --> -----
2 : AREEK --> ar---
3 : BURMA --> burmA
4 : RUMBA --> rumbA
5 : UMBRA is correct!


## Here's the normal wordle game ...

In [9]:
import pathlib
import random
words = pathlib.Path('wordle_words.txt').read_text('utf-8').split('\n')

secret_word = random.choice(words)
keep, eliminate, guesses = set(),set(), 0
print("secret word has been chosen. Your turn...")
options = words[:]
while 1:
    guess = input(">>>")
    if guess == 'q':
        break
    if guess == 'r':
        guess = random.choice(options)
    if guess == 'h':
        print(options[:12])
        continue

    guesses += 1
    if guess not in words:
        print(guess, "is not a word")
        continue
        
    if guess == secret_word:
        print(f"{guess} is correct!")
        break
    
    options.remove(guess)
    word = compare(guess, secret_word)
    
    keep.update({c.upper() for c in word if c != '-'})
    eliminate.update({c for c in guess if c not in keep})

    options = [w for w in options if not any(c in w for c in eliminate)]
    if keep:
        options = [w for w in options if keep.issubset(set(w))]
    options = [w for w in options if could_match(word,w)]
    print(guesses,":", guess, "-->", word)


secret word has been chosen. Your turn...
1 : SWIFT --> --i-t
2 : TIMMI --> TI---
 is not a word
['TIANA', 'TIANG', 'TIARA', 'TIBBY', 'TIBBU', 'TIBEY', 'TIBER', 'TIBET', 'TIBIA', 'TIBUR', 'TICAL', 'TICCA']
4 : TIBBY --> TI---
5 : TICCA --> TI--A
['TIANA', 'TIARA', 'TIENA', 'TIGUA', 'TIKKA', 'TILDA', 'TILIA', 'TILLA', 'TINEA', 'TINIA', 'TINTA', 'TIOGA']
6 : TIOGA --> TI--A
7 : TINEA --> TI--A
8 : TIARA --> TI--A
9 : TILDA --> TIL-A
TILLA is correct!
